# 第8回講義 宿題

## 課題. Tensorflowを用いて, CIFAR-10を畳み込みニューラルネットワーク(CNN)で学習せよ

### 注意

- homework関数を完成させて提出してください
    - 訓練データはtrain_X, train_y, テストデータはtest_Xで与えられます
    - train_Xとtrain_yをtrain_X, train_yとvalid_X, valid_yに分けるなどしてモデルを学習させてください
    - test_Xに対して予想ラベルpred_yを作り, homework関数の戻り値としてください\
- pred_yのtest_yに対する精度(F値)で評価します
- 全体の実行時間がiLect上で60分を超えないようにしてください
- homework関数の外には何も書かないでください (必要なものは全てhomework関数に入れてください)
- 解答提出時には Answer Cell の内容のみを提出してください

**`tf` の以下のモジュール及び `keras` はこの回では使用できないように制限されています. 注意してください.**

```python
tf.app
tf.compat
tf.contrib
tf.erros
tf.gfile
tf.graph_util
tf.image
tf.layers
tf.logging
tf.losses
tf.metrics
tf.python_io
tf.resource_loader
tf.saved_model
tf.sdca
tf.sets
tf.summary
tf.sysconfig
tf.test
```

次のセルのhomework関数を完成させて提出してください

# Answer Cell

In [1]:
def homework(train_X, train_y, test_X):
    rng = np.random.RandomState(1234)
    random_state = 42
    train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y,test_size=0.2, random_state=42)
    flip_train_X = train_X[:, :, ::-1, :]
    padded = np.pad(train_X, ((0, 0), (4, 4), (4, 4), (0, 0)), mode='constant')
    crops = rng.randint(8, size=(len(train_X), 2))
    cropped_train_X = [padded[i, c[0]:(c[0]+32), c[1]:(c[1]+32), :] for i, c in enumerate(crops)]
    cropped_train_X = np.array(cropped_train_X)
    train_X = np.concatenate((train_X, flip_train_X))
    train_X = np.concatenate((train_X,  cropped_train_X ))
    temp = train_y
    train_y = np.concatenate((train_y, train_y))
    train_y = np.concatenate((train_y, temp))
    import matplotlib.pyplot as plt

    # グラフをipython notebook内に表示
    %matplotlib inline
    def gcn(x):
        mean = np.mean(x, axis=(1, 2, 3), keepdims=True)
        std = np.std(x, axis=(1, 2, 3), keepdims=True)
        return (x - mean)/std
    
    class ZCAWhitening:
        def __init__(self, epsilon=1e-4):
            self.epsilon = epsilon
            self.mean = None
            self.ZCA_matrix = None

        def fit(self, x):
            x = x.reshape(x.shape[0], -1)
            self.mean = np.mean(x, axis=0)
            x -= self.mean
            cov_matrix = np.dot(x.T, x) / x.shape[0]
            A, d, _ = np.linalg.svd(cov_matrix)
            self.ZCA_matrix = np.dot(np.dot(A, np.diag(1. / np.sqrt(d + self.epsilon))), A.T)

        def transform(self, x):
            shape = x.shape
            x = x.reshape(x.shape[0], -1)
            x -= self.mean
            x = np.dot(x, self.ZCA_matrix.T)
            return x.reshape(shape)
    
    class BatchNorm:
        def __init__(self, shape, epsilon=np.float32(1e-5)):
            self.gamma = tf.Variable(np.ones(shape, dtype='float32'), name='gamma')
            self.beta  = tf.Variable(np.zeros(shape, dtype='float32'), name='beta')
            self.epsilon = epsilon

        def f_prop(self, x):
            if len(x.get_shape()) == 2:
                mean, var = tf.nn.moments(x, axes=0, keepdims=True)
                std = tf.sqrt(var + self.epsilon)
            elif len(x.get_shape()) == 4:
                mean, var = tf.nn.moments(x, axes=(0,1,2), keep_dims=True)
                std = tf.sqrt(var + self.epsilon)
            normalized_x = (x - mean) / std
            return self.gamma * normalized_x + self.beta
        
    class Conv:
        def __init__(self, filter_shape, function=lambda x: x, strides=[1,1,1,1], padding='VALID'):
            # Xavier
            fan_in = np.prod(filter_shape[:3])
            fan_out = np.prod(filter_shape[:2]) * filter_shape[3]
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(fan_in + fan_out)),
                            high=np.sqrt(6/(fan_in + fan_out)),
                            size=filter_shape
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros((filter_shape[3]), dtype='float32'), name='b') # バイアスはフィルタごと
            self.function = function
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            u = tf.nn.conv2d(x, self.W, strides=self.strides, padding=self.padding) + self.b
            return self.function(u)
        
    class Pooling:
        def __init__(self, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID'):
            self.ksize = ksize
            self.strides = strides
            self.padding = padding

        def f_prop(self, x):
            return tf.nn.max_pool(x, ksize=self.ksize, strides=self.strides, padding=self.padding)
    
    class Flatten:
        def f_prop(self, x):
            return tf.reshape(x, (-1, np.prod(x.get_shape().as_list()[1:])))
        
    class Dense:
        def __init__(self, in_dim, out_dim, function=lambda x: x):
            # Xavier
            self.W = tf.Variable(rng.uniform(
                            low=-np.sqrt(6/(in_dim + out_dim)),
                            high=np.sqrt(6/(in_dim + out_dim)),
                            size=(in_dim, out_dim)
                        ).astype('float32'), name='W')
            self.b = tf.Variable(np.zeros([out_dim]).astype('float32'))
            self.function = function

        def f_prop(self, x):
            return self.function(tf.matmul(x, self.W) + self.b)
        
    class Activation:
        def __init__(self, function=lambda x: x):
            self.function = function

        def f_prop(self, x):
            return self.function(x)
        
    layers = [ # (縦の次元数)x(横の次元数)x(チャネル数)
        Conv((3, 3, 3, 32)), # 32x32x3 -> 30x30x32
        BatchNorm((30, 30, 32)),
        Activation(tf.nn.relu),
        Pooling((1, 2, 2, 1)), # 30x30x32 -> 15x15x32
        Conv((3, 3, 32, 64)), # 15x15x32 -> 13x13x64
        BatchNorm((13, 13, 64)),
        Pooling(((1, 2, 2, 1))), # 13x13x64 -> 6x6x64
        Conv((3, 3, 64, 128)), # 6x6x64 -> 4x4x128
        BatchNorm((4, 4, 128)),
        Activation(tf.nn.relu),
        Pooling((1, 2, 2, 1)), # 4x4x128 -> 2x2x128
        Flatten(),
        Dense(2*2*128, 256, tf.nn.relu),
        Dense(256, 10, tf.nn.softmax)
    ]

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    t = tf.placeholder(tf.float32, [None, 10])

    def f_props(layers, x):
        for layer in layers:
            x = layer.f_prop(x)
        return x

    y = f_props(layers, x)

    cost = -tf.reduce_mean(tf.reduce_sum(t * tf.log(tf.clip_by_value(y, 1e-10, 1.0)), axis=1))
    train = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

    valid = tf.argmax(y, 1)
    
    #zca = ZCAWhitening()
    #zca.fit(gcn(train_X))
    #zca_train_X = zca.transform(gcn(train_X))
    #zca_valid_X = zca.transform(gcn(valid_X))
    #zca_train_y = train_y[:]
    #zca_test_X = zca.transform(gcn(test_X))
    
    n_epochs = 50
    batch_size = 100
    n_batches = train_X.shape[0]//batch_size

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)
    y_train = []
    y_valid = []
    for epoch in range(n_epochs):
        train_X, train_y = shuffle(train_X, train_y, random_state=random_state)
        valid_cost=0
        for i in range(n_batches):
            start = i * batch_size
            end = start + batch_size
            _,_cost=sess.run([train, cost], feed_dict={x: train_X[start:end], t: train_y[start:end]})
            valid_cost += _cost
        xx_cost = sess.run([cost], feed_dict={x: valid_X, t: valid_y})
        y_train.append(valid_cost/(n_batches+1))
        y_valid.append(xx_cost)
        print('EPOCH:: %i, Validation cost: %.3f' % (epoch + 1, valid_cost))
    plt.figure(1)
    plt.plot(y_train)  
    plt.plot(y_valid)
    plt.show()
    pred_y_m = sess.run([valid], feed_dict={x: test_X})
    pred_y=pred_y_m[0]
    sess.close()
    return pred_y

- 以下のvalidate_homework関数を用いてエラーが起きないか動作確認をして下さい。
- 提出に際して、以下のscore_homework関数で60分で実行が終わることを確認して下さい。
- 評価は以下のscore_homework関数で行われますが、random_stateの値は変更されます。

# Checker Cell (for student)

In [ ]:
import sys

from keras.datasets import cifar10
from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf

del [
    tf.app,
    tf.compat,
    tf.contrib,
    tf.errors,
    tf.gfile,
    tf.graph_util,
    tf.image,
    tf.layers,
    tf.logging,
    tf.losses,
    tf.metrics,
    tf.python_io,
    tf.resource_loader,
    tf.saved_model,
    tf.sdca,
    tf.sets,
    tf.summary,
    tf.sysconfig,
    tf.test
]

sys.modules['keras'] = None

def load_cifar():
    (cifar_X_1, cifar_y_1), (cifar_X_2, cifar_y_2) = cifar10.load_data()

    cifar_X = np.r_[cifar_X_1, cifar_X_2]
    cifar_y = np.r_[cifar_y_1, cifar_y_2]

    cifar_X = cifar_X.astype('float32') / 255
    cifar_y = np.eye(10)[cifar_y.astype('int32').flatten()]

    train_X, test_X, train_y, test_y = train_test_split(cifar_X, cifar_y,
                                                        test_size=10000,
                                                        random_state=42)

    return (train_X, test_X, train_y, test_y)

def validate_homework():
    train_X, test_X, train_y, test_y = load_cifar()

    # validate for small dataset
    train_X_mini = train_X[:100]
    train_y_mini = train_y[:100]
    test_X_mini = test_X[:100]
    test_y_mini = test_y[:100]

    pred_y = homework(train_X_mini, train_y_mini, test_X_mini)
    print(f1_score(np.argmax(test_y_mini, 1), pred_y, average='macro'))

def score_homework():
    train_X, test_X, train_y, test_y = load_cifar()
    pred_y = homework(train_X, train_y, test_X)
    print(f1_score(np.argmax(test_y, 1), pred_y, average='macro'))

Using TensorFlow backend.


In [ ]:
#validate_homework()
score_homework()

EPOCH:: 1, Validation cost: 1490.672
